In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qg-bert-en-dec/model_4
/kaggle/input/qg-bert-en-dec/model_3
/kaggle/input/qg-bert-en-dec/model_1
/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json
/kaggle/input/stanford-question-answering-dataset/train-v1.1.json


In [2]:
import logging
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
from transformers import BertConfig
import json
import spacy


def get_tokenizer(model_type_path):
    tok = BertTokenizer.from_pretrained(model_type_path, do_lower_case=True)
    return tok

def get_mask_ids(tokens, max_seq_length):
    """attention Mask id for padding 1 for original 0 for padded"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segment_ids(tokens, max_seq_length):
    """Segments id : 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
                current_segment_id = 1
    assert current_segment_id ==1
    return segments + [0] * (max_seq_length - len(tokens))

def get_token_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids



def process_input(text, ans, ques, tokenizer, max_seq_len):
    text_token = tokenizer.tokenize(text)
    ans_token= tokenizer.tokenize(ans)
    ques_token= tokenizer.tokenize(ques)

    if len(text_token) > max_seq_len-3 -len(ans_token):
        text_token = text_token[:max_seq_len- 3-len(ans_token)]

    return text_token, ans_token, ques_token
    
def pad_ques(ques, max_seq_len, padding =0):
    if len(ques)> max_seq_len:
        raise IndexError("len of ques {} greater than max_seq_len{}".format(len(ques), max_seq_len))

    req_len= max_seq_len- len(ques)
    ques += [padding]* req_len

    return ques



def convert_to_input(text, ans, ques, tokenizer, max_seq_len):
    text_token, ans_token, ques_token= process_input(text,ans,ques, tokenizer, max_seq_len)

    allToken= ["[CLS]"] + text_token  + ["[SEP]"] + ans_token + ["[SEP]"]
    ques_token= ["[CLS]"] + ques_token  + ["[SEP]"]
    ids= get_token_ids(allToken, tokenizer, max_seq_len)
    mask_ids = get_mask_ids(allToken, max_seq_len)
    segment_ids = get_segment_ids(allToken, max_seq_len)

    que_ids= get_token_ids(ques_token, tokenizer, len(ques_token))

    que_ids= pad_ques(que_ids, max_seq_len) #CR padding
    assert len(que_ids)== max_seq_len
    return ids, mask_ids, segment_ids, que_ids


def generate_context_list(text):
    ''' Extract chunks with spacy from text paragraph
    '''
    nlp= spacy.load('en_core_web_sm') #this will download spacy english core model
    nlpDoc= nlp(text)
    lst= []
    for chunk in nlpDoc.noun_chunks:
        lst.append(chunk)

    return lst







class QGDataset(Dataset):
    def __init__(self, args):
        super().__init__()
        self.args= args
        self.examples= []
        self.tokenizer= get_tokenizer(args.bert_model)
        self.max_seq_len= self.args.max_seq_len

        if args.squad_path== None and args.inferenceMode == False:
            raise ValueError("invaild path to squad 1.0 data or wrong inference mode")

        if args.squad_path != None:
            with open(args.squad_path) as f:
                json_data= json.load(f)
                json_data = json_data['data']

            for data in json_data:
                for para in data['paragraphs']:
                    con = para['context']
                    qas= para['qas']
                    for xs in qas:
                        cur_ans= xs['answers'][0]['text']
                        cur_ans_offset= xs['answers'][0]['answer_start']
                        que= xs['question']
                        ex= {
                            'text': con,
                            'ans': cur_ans,
                            'ans_offset': cur_ans_offset,
                            'ques': que
                        }
                        self.examples.append(ex)
            del json_data
        
        else:
            if args.inferenceFile == None:
                raise ValueError("wrong file for inference")
            
            with open(args.inferenceFile) as f:
                json_data= json.load(f)
            
            for item in json_data:
                paragraph= None
                contextList= None
                if 'paragraph' in item.keys():
                    paragraph = item['paragraph']
                
                else:
                    raise KeyError("no text para graph is found. worong format of inference file")
                
                if 'context_list' in  item.keys():
                    contextList= item['context_list']
                
                else:
                    contextList = generate_context_list(paragraph)
                
                for context in contextList:
                    cur_ex= {
                        'text': paragraph,
                        'ans': context,
                        'ques': "a dummy ques to avoid None error"
                    }

                    self.examples.append(cur_ex)
                

        if args.occu:
            self.examples= self.examples[:args.occu]


    def __len__(self):
        return len(self.examples)

    def __getitem__(self,idx):
        cur_ex= self.examples[idx]
        ids, mask_id, seg_id, ques= convert_to_input(cur_ex['text'], cur_ex['ans'], cur_ex['ques'], self.tokenizer, self.max_seq_len)

        exm= {
            'ids': torch.tensor(ids, dtype= torch.long),
            'mask_ids': torch.tensor(mask_id, dtype= torch.long),
            'segment_ids': torch.tensor(seg_id, dtype= torch.long),
            'ques': torch.tensor(ques, dtype= torch.long)
        }
#         print(exm)
        return exm
    


In [4]:
from nltk.translate.bleu_score import sentence_bleu

In [5]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def get_blue_score(orig, pre):
#     print("orignal blue ->>>>:" , orig,"\n Predicted blue->>>>", pre)
    orig_tok= orig.split()
    pre_tok= pre.split()[:len(orig_tok)]
#     print("orignal blue ->>>>:" , orig_tok,"\n Predicted blue->>>>", pre_tok)
    ref= [orig_tok]
    score= sentence_bleu(ref, pre_tok)
    return score

In [6]:
# from arguments import inferenceArgs
# from process import QGDataset
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


def inference(args):
    if torch.cuda.is_available():
        device= torch.device("cuda")  
    else:
        device= torch.device("cpu")
        print("using CPU as device, may cause Out of memory error")

    evalData= QGDataset(args)
    batch_size = min(16,evalData.__len__())
    ignore_label= -1 #check
    worker=0
    model= torch.load(args.infereceModelPath)
    model.to(device)
    
    model.eval()
    
    tokenizer = evalData.tokenizer
    vocab_size = tokenizer.vocab_size
    evalDataLoader = DataLoader(evalData,batch_size=batch_size, num_workers= worker)
    total_acc= AverageMeter()
    tdl = tqdm(evalDataLoader, total= len(evalDataLoader))
    for idx,batch in enumerate(tdl):
    
        ids = batch['ids'].to(device, dtype=torch.long)
        mask_ids = batch['mask_ids'].to(device, dtype=torch.long)
        seg_ids = batch['segment_ids'].to(device, dtype=torch.long)
        ques = batch['ques'].to(device, dtype=torch.long)
        with torch.no_grad():
            logits= model(
                input_ids= ids,
                attention_mask= mask_ids,
                decoder_input_ids= ids,
                # decoder_inputs_embeds= model.get_input_embeddings().weight,
                token_type_ids= seg_ids,
                masked_lm_labels = None
            )
#         print(logits)
        logits= logits[0]
        logits= logits.view(-1, vocab_size)
        # orig_ques= ques.view(-1)
        logits = logits.detach().cpu().numpy()
        pred_ques = np.argmax(logits, axis=1).flatten().squeeze()
        pred_ques = np.reshape(pred_ques,(batch_size,-1))
        for i in range(ids.shape[0]):
            cur_pred_ques= list(pred_ques[i])
            try:
                cur_len= cur_pred_ques.index(102) # find first sep token
            except ValueError:
                cur_len= len(cur_pred_ques)-1

            cur_pred_ques = cur_pred_ques[:cur_len+1]
            cur_pred_ques= tokenizer.decode(cur_pred_ques, skip_special_tokens=True)
            cur_orignal_ques= tokenizer.decode(list(ques[i]), skip_special_tokens=True)
            ca= get_blue_score(cur_orignal_ques, cur_pred_ques)
            total_acc.update(ca)
            tdl.set_postfix(status= 'valid',accu= total_acc.avg)
#             print("orignal ->>>>:" , cur_orignal_ques,"\n Predicted->>>>", cur_pred_ques)
            
#             print(cur_pred_ques)

    print("avga accu: ", total_acc.avg)
    


# if __name__ == "__main__":
#     inference(inferenceArgs)

In [7]:

from transformers import BertConfig 

bert_model= 'bert-base-uncased'
max_seq_len =256

class trainDataArgs:
    bert_model= bert_model
    max_seq_len = max_seq_len
    squad_path= '../input/stanford-question-answering-dataset/train-v1.1.json' #CR
    inferenceMode= False  # T/F for inference
    inferenceFile= None  # if nferenceMode = True: a path to json file consisting of list of dict with keys 'paragraph' as mandatory key and 'context_list' as option  [{'paragraph':' a string', 'context_list': ['list' , 'of', 'context', 'i.e answers'] }, {}, . . .]

    occu = 30000

class validDataArgs:
    bert_model= bert_model
    max_seq_len =max_seq_len
    squad_path= '../input/stanford-question-answering-dataset/dev-v1.1.json' #CR
    inferenceMode= False  # T/F for inference
    inferenceFile= None  # if nferenceMode = True: a path to json file consisting of list of dict with keys 'paragraph' as mandatory key and 'context_list' as option  [{'paragraph':' a string', 'context_list': ['list' , 'of', 'context', 'i.e answers'] }, {}, . . .]
    occu= 8000

class inferenceArgs:
    bert_model= bert_model
    max_seq_len =max_seq_len
    squad_path= '/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json' #CR
    inferenceMode= False  # T/F for inference
    inferenceFile= None  # CR
    # if nferenceMode = True: a path to json file consisting of list of dict with keys 'paragraph' as mandatory key and 'context_list' as option  [{'paragraph':' a string', 'context_list': ['list' , 'of', 'context', 'i.e answers'] }, {}, . . .]
    occu= None
    infereceModelPath= '/kaggle/input/qg-bert-en-dec/model_4'


class trainingConfig:
    bert_model = bert_model
    bert_config = BertConfig.from_pretrained(bert_model)
    max_seq_len= max_seq_len
    train_batch_size= 16
    valid_batch_size= 16
    ignore_label =0
    num_workers=0
    epochs =3
    learningRate = 5e-5
    save_dir = '/kaggle/working/'

In [8]:
inference(inferenceArgs)

  0%|          | 0/661 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
  0%|          | 1/661 [00:01<21:07,  1.92s/it, accu=0.64, status=valid]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
  2%|▏         | 10/661 [00:12<12:17,  1.13s/it, accu=0.471, status=valid]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
100%|██████████| 661/661 [12:14<00:00,  1.11s/it, accu=0.369, status=valid]

avga accu:  0.3691226051927911
